In [0]:
!pip install -q sk-video
!pip install -q pims
import imutils
import json
import cv2
import pims
import matplotlib.pyplot as plt
import skvideo.io
import skvideo
import numpy as np
import math
from shapely.geometry import Polygon
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
# read video from gdrive
base_path = '/content/gdrive/My Drive/videos/'
video_name = base_path + 'blur_night.MOV'
images = pims.Video(video_name) 

# get a resize version of frames
images_resize = []
for i,img in enumerate(images):
  images_resize.append(imutils.resize(img, width=500))

max_frame = len(images)

# get image dimensions
original_shape = images[0].shape
img_0 = imutils.resize(images[0].copy(), width=500)
image_width = img_0.shape[1]
image_height = img_0.shape[0]

# relation pixels to meters, and margin for max natural movement
pixel_meter = original_shape[1]/12 # baseline meters
x_max_movement = 200
y_max_movement = 120

# value used to get differeces when detecting player
delta = 3

# Negative values is left, positive right
consistent_direction = 0
delta_direction = 7

# Yellow Range
yellowLower = (50, 50, 100)
yellowUpper = (90, 100, 200)

# Inicialize for ball detection
num_previous_contours = 6
num_my_side_frames = 24
num_other_side_frames = 24

# hardcode court zones
y_netline = 330 
y_baseline = 400 
left_center_x = 400
right_center_x = 900

# color setup
color = (0, 0, 255)
color_down = (128,128,255)
color_up = (255,128,128)


In [0]:
# inicialize for player detection
frames_out = []
list_x = []
list_y = []
player_position_x = []
player_position_y = []
total_movement_x = 0

# Inicialize variables for ball detection
previous_contours = [[] for i in range(num_previous_contours)]
ball_candidates_contours = [[] for i in range(num_previous_contours)]
discard_candidates_contours = [[] for i in range(num_previous_contours)]
i_previous_contours = 0;
cnt_frames = 0
i_last_best_trajectories = 0
last_best_trajectories = [[] for i in range(num_my_side_frames)]
x_ball = 0

# Auxiliar conters for stroke detection
big_ball_cnt = 0
sum_x_position_player = 0
color = None

# Frame range from movie
init_frame = 0 
end_frame = max_frame - delta

text = ''

writer = skvideo.io.FFmpegWriter(base_path + 'result.mp4')

for i in range(init_frame+delta,end_frame):
  
  #Current frame
  frame = images[i].copy()
  i_image = i
    
  # Detect player and calculate it contour
  frame_threahold = image_diff_delta(images_resize,i,delta)
  player_contour = get_contour_player(frame_threahold)

  # Detect ball
  
  # Get yellow mask
  color_mask = get_color_mask(frame,yellowLower,yellowUpper)
  
  # Get FOM mask
  frame_plus_minus = image_diff_plus_minus(images,i)
  
  # Mix both mask and get contours and trayectories
  frame_threahold = mix_mask(color_mask,frame_plus_minus)
  ball_candidates_contours = get_ball_contours(frame_threahold,previous_contours,i_previous_contours,y_baseline)
  best_trajectory = get_best_trajectory(previous_contours,i_previous_contours,margin_x=50,margin_y=20,max_trajectory=2) 

  # draw info in frame
  total_movement_x, consistent_direction = draw_player(frame,player_contour,list_x,list_y,player_position_x,player_position_y,
      total_movement_x,consistent_direction,delta_direction,original_shape,image_width,image_height,pixel_meter)
  paint_player_info(frame,total_movement_x,player_position_x,player_position_y)

  # draw detected contours
  paint_trajectory(frame,best_trajectory,color_down,color_up) 
  last_best_trajectories[i_last_best_trajectories] = best_trajectory 
  ball_candidate = paint_circle_contours(frame,ball_candidates_contours,best_trajectory)
 
  # if exists a contour is bigger than 3.8, we start count num_my_side_frames frames
  if len(best_trajectory) and len(ball_candidate) and ball_candidate[2] > 3.8:   
    if big_ball_cnt == 0:
      big_ball_cnt += 1
      
  # Count frames after big contour detected
  if not big_ball_cnt == 0:
    big_ball_cnt += 1
  
  # If any big contour detected
  if big_ball_cnt > 0:    
    
    sum_x_position_player += player_position_x[-1]
    get_trajectory_bigest_ball(last_best_trajectories)
    
    # If num_my_side_frames has pass, we try to guess what type of stroke is
    if big_ball_cnt > num_my_side_frames:

      big_ball_cnt = - num_other_side_frames
      avg_player_x = sum_x_position_player /num_my_side_frames
      
      # Stroke detected outside center zone
      if  avg_player_x > right_center_x:
        text = 'FOREHAND STROKE DETECTED'
        color = (255,128,128)
      elif avg_player_x < left_center_x:
        text = 'BACKHAND STROKE DETECTED'
        color = (120, 255, 120)
      else: # Stroke detected inside center zone
        x_ball = get_x_in_my_side(last_best_trajectories)
        i,x_ball_bigest = get_trajectory_bigest_ball(last_best_trajectories)
        
        # compare ball position and player position
        if x_ball_bigest[0] > avg_player_x:
          text = 'FOREHAND CENTRAL STROKE DETECTED'
          color = (255,28,128)
        else:
          text = 'BACKHAND CENTRAL STROKE DETECTED'
          color = (0, 255, 120)                    
        
      sum_x_position_player = 0
    else: # detecting frames in process
      text = 'DETECTING STROKE ({}).'.format(big_ball_cnt)
      color = (120, 120, 255)
      for i in range(big_ball_cnt // 3):
        text += '.'
      
  cv2.putText(frame,text,(300, 600),cv2.FONT_HERSHEY_SIMPLEX,1,color,2)

  i_previous_contours = (i_previous_contours + 1) % num_previous_contours  
  i_last_best_trajectories = (i_last_best_trajectories +1) % num_my_side_frames
  
  writer.writeFrame(frame)

writer.close()


In [0]:
# Draw lines in a image
def draw_lines(img,line,widht =1,color = (0, 255, 0)): 
  if len(line) > 1:
    for i in range(1,len(line)):
      cv2.line(img,line[i-1],line[i], color, widht+i)  

# Coordinates conversion for different size images      
def d2d(x,y,original_shape,image_width,image_height):
  return (x * original_shape[1] // image_width, y * original_shape[0] // image_height)

# Calculate a paint in a frame player position
def draw_player(image,player_contour,list_x,list_y,player_position_x,player_position_y,total_movement_x, consistent_direction,
                delta_direction,original_shape,image_width,image_height,pixel_meter):
  
  # Any contour
  if player_contour != []:
    
    # Calculate x,y of contour detected
    (x, y, w, h) = cv2.boundingRect(player_contour)
    
    p1 = d2d(x,y,original_shape,image_width,image_height)
    p2 = d2d(x + w, y + h,original_shape,image_width,image_height)
    
    x = p1[0]
    y = p1[1]
    w = p2[0]-p1[0]
    h = p2[1]-p1[1]
    
    x2 = (x*2 + w)//2
    y2 = y+h
    
    # If new position is close to previus position, we add it
    if ((len(player_position_x) > 0 and 
        abs(player_position_x[-1] - x2) < x_max_movement and abs(player_position_y[-1] - y2) < y_max_movement))\
        or len(player_position_x) == 0:

      list_x.insert(0,x2)
      list_y.insert(0,y2)

    # When there are more than five position, player position is calculate from median of las position detected
    if len(list_x) > 5:
      x_median = int(np.median(list_x))
      y_median = int(np.median(list_y))

      list_x.pop()
      list_y.pop()
      player_position_x.append(x_median)
      player_position_y.append(y_median)
      
      # Draw a circle in player position
      cv2.circle(image,(player_position_x[-1] ,player_position_y[-1]),10,(200, 200,120),-1)
      
      # Calculate movement direction
      if len( player_position_x) > 1:

        direction_aux = player_position_x[-1] - player_position_x[-2]
        if direction_aux > 0: #Right
          if consistent_direction >= 0: # Same Direction
            if consistent_direction > delta_direction:            
              total_movement_x += abs(direction_aux) / pixel_meter
            else:
              consistent_direction += 1
          else:
            consistent_direction = 0
        elif direction_aux < 0: #Left
          if consistent_direction <= 0: # Same Direction
            if abs(consistent_direction) > delta_direction:
              total_movement_x += abs(direction_aux) / pixel_meter
            else:
              consistent_direction -= 1
          else:
            consistent_direction = 0
            
  return total_movement_x, consistent_direction

# Draw distance covered by player, and x,y position in pixels
def paint_player_info(image,total_movement_x,player_position_x,player_position_y):           

  if len( player_position_x) > 1:
    #draw meter covered by player (left top corner) and player position in pixeles (left bottom corner)
    cv2.putText(frame,"Distance covered: {} meters".format(round(total_movement_x,2)),(10, 20),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0, 128),2)
    cv2.putText(frame,"Player Position: {} pixel".format(str(round(player_position_x[-1])) + ',' + str(round(player_position_y[-1]))),(10,frame.shape[0]-10),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),1)
    

# Draw ball contour
def paint_circle_contours(image, contours,best_trajectory):

  for c in contours:
    (xc, yc, radius) = c
  
  c = get_closest_contour(best_trajectory,contours)
  if len(c)> 0:
    (xc, yc, radius)= c
    cv2.circle(image, (int(xc), int(yc)), int(radius),(255, 255, 0), 3)
  return c
  
# draw trajectory
def paint_trajectory(image,trajectory,color_down,color_up):

  if len(trajectory) > 2:
    color = color_down
    if line_go_up(trajectory):
      color = color_up
    line = []
    for e in trajectory:
      line.append((e[0],e[1])) 

    draw_lines(image,line,3,color)
    
  

In [0]:
# Return true if line go up
def line_go_up(line):
  return line[0][1] - line[-1][1] > 0
  
# Get Distance between two points
def get_distance(p1,p2):
  return math.sqrt(((p2[0]-p1[0])**2) + ((p2[1]-p1[1])**2))
  
# Return contours have a radius between min_radius and max_radius
def get_contours_by_size(contours,min_radius,max_radius):
  
  big_contours = 0
  filtered_contours = []
  
  for c in contours:  
    ((xc, yc), radius) = cv2.minEnclosingCircle(c)

    # only proceed if the radius meets a minimum size
    if (radius > min_radius) & (radius < max_radius):
      filtered_contours.append((xc, yc, radius))
      big_contours += 1
  
  return filtered_contours

# Group close contour and eliminate those higher than y_baseline (those are not normally the ball)
def group_contours(contours,margin,y_baseline):

  intersected = True

  # Delete contours out baseline
  i = 0
  while i < len(contours):
    if contours[i][1] > y_baseline:
      contours.pop(i)
    else:
      i += 1
  
  # Group contours
  while intersected:
    intersected = False
    i = 0
    while i < (len(contours)-1):
      j = i+1
      while j < len(contours):
        (xc1, yc1, radius1) = contours[i]
        (xc2, yc2, radius2) = contours[j]
        if abs(xc1-xc2) < (radius1 + radius2 + margin) and abs(yc1-yc2) < (radius1 + radius2 + margin):
          if radius1 < radius2:
            xc1 = xc2
            yc1 = yc2
            radius1 = (radius1*2) + radius2
          else:
            radius1 = (radius2*2) + radius1
            
          contours[j] = (xc1,yc1,radius1)
          contours.pop(i)
          intersected = False
          i = 0
          j = 1
        else:
          j += 1
      i += 1
        
# Get the closer contour (if any) to a trajectory, by a margin
def get_closest_contour(trajectory,contours,margin=50):
   
  closest_contour = []
  for i,c in enumerate(contours):
    for trj in trajectory:
      distance = get_distance(c,trj)
      if distance < margin:
        margin = distance
        closest_contour = c
  return closest_contour
  
# Get the biggest contour in the trajectories detected (most probably the ball)
def get_trajectory_bigest_ball(last_best_trajectories):
  max_radius = 0
  max_ball = []
  for i,trj in enumerate(last_best_trajectories):
    if len(trj) > 0 and trj[-1][2] > max_radius:
      max_radius = trj[-1][2]
      max_ball = trj[-1]
  return i,max_ball
    

# Combine all contours detected to build posible trajectories
def get_possible_trayectories(prev_contours,i_prev_contours,margin_x=20,margin_y=50):
  
  trajectories = []
  level = len(prev_contours)
  # for every previus contour level
  for i in range(i_prev_contours +1,len(prev_contours)+ i_prev_contours +1):
    
    trajectories_level = []
    # for every contour
    i = i % len(prev_contours) # avoid index out exception
    for c in prev_contours[i]:
    
      # for every current trajectory
      for j in range(len(trajectories)):       
        for k in range(len(trajectories[j])):
          point = trajectories[j][k]

          if abs(c[0]+c[2]-point[0]) < margin_x and abs(c[1]+c[2]-point[1]) < margin_y:
            trj_aux = trajectories[j].copy()
            trj_aux.append((int(c[0]),int(c[1]),c[2]))
            trajectories_level.append(trj_aux)
            break
          
      trajectories_level.append([(int(c[0]),int(c[1]),c[2])])
    trajectories.extend(trajectories_level)
    level -= 1
    
  return trajectories

# Get avg x of a trajectory
def get_x_in_my_side(trajectory):
  sum_x = 0
  total = 0
  for trj in trajectory:
    for c in trj:
      sum_x += c[0]
      total += 1
  if total == 0:
    return -1
  return sum_x // total

# get contour player
def get_contour_player(image):
  
  cnts = cv2.findContours(image, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)

  #loop over the contours
  player_contour = []
  countour_player = 0

  for c in cnts:  
    # if the contour is too small, ignore it
    if cv2.contourArea(c) < 700:
      continue

    contour_aux = cv2.contourArea(c)
    if contour_aux > countour_player:
      countour_player = contour_aux
      player_contour = c

  return player_contour

# Grab and store balls contours
def get_ball_contours(frame,previous_contours,i_previous_contours,y_baseline):

  cnts = cv2.findContours(frame, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)

  previous_contours[i_previous_contours] = get_contours_by_size(cnts,0.1,15)
  group_contours(previous_contours[i_previous_contours],3,y_baseline)

  return previous_contours[i_previous_contours]

# Return the best trajectory as the one owns the biggest contour (most probably the ball)
def get_best_trajectory(previous_contours,i_previous_contours,margin_x=50,margin_y=20,max_trajectory = 0):
  
  trajectories = get_possible_trayectories(previous_contours,i_previous_contours,margin_x,margin_y)
  best_trajectory = []
  best_trajectories = []
  len_contours = len(previous_contours)
  
  for trj in trajectories:
    if len(trj) > max_trajectory or len(trj) == len_contours:
      max_trajectory = len(trj)
      best_trajectories.append(trj)
  
  # This only return a best if there is any contours close enough a trajectory
  if len(previous_contours[i_previous_contours]):
    max_radius = 0
    for trj in best_trajectories:
      c = get_closest_contour(trj,previous_contours[i_previous_contours])
      if len(c)>0 and c[2] > max_radius:
        best_trajectory = trj
  
  return best_trajectory
  

In [0]:
# Get differences mask to detect the player
def image_diff_delta(images,i,delta):


  frame = images[(i-delta)]
  
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  
  gray = cv2.GaussianBlur(gray, (21, 21), 0)
  avg = gray.astype("float")

  frame = images[i]

  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  gray = cv2.GaussianBlur(gray, (21, 21), 0).copy()
  
  cv2.accumulateWeighted(gray, avg, 0.5)
  frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))

  thresh = cv2.threshold(frameDelta, 5, 255,cv2.THRESH_BINARY)[1]
  thresh = cv2.dilate(thresh, None, iterations=2)
  
  return thresh
  


# Uses to ball detection
def image_diff_plus_minus(images,i):
  
  im_im1 = cv2.cvtColor(images[i-1], cv2.COLOR_BGR2GRAY)
  im_i = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
  im_ip1 = cv2.cvtColor(images[i+1], cv2.COLOR_BGR2GRAY)
  
  # differences
  delta_plus = cv2.absdiff(im_i, im_im1)
  delta_0 = cv2.absdiff(im_ip1, im_im1)
  delta_minus = cv2.absdiff(im_i,im_ip1)
 
  # threadhold
  sp = cv2.meanStdDev(delta_plus)
  sm = cv2.meanStdDev(delta_minus)
  s0 = cv2.meanStdDev(delta_0)

  th = [
      sp[0][0, 0] + 3 * math.sqrt(sp[1][0, 0]),
      sm[0][0, 0] + 3 * math.sqrt(sm[1][0, 0]),
      s0[0][0, 0] + 3 * math.sqrt(s0[1][0, 0]),
  ]

  ret, dbp = cv2.threshold(delta_plus, th[0], 255, cv2.THRESH_BINARY)
  ret, dbm = cv2.threshold(delta_minus, th[1], 255, cv2.THRESH_BINARY)
  ret, db0 = cv2.threshold(delta_0, th[2], 255, cv2.THRESH_BINARY)
  
  detect = cv2.bitwise_and(cv2.bitwise_and(dbp, dbm), cv2.bitwise_not(db0))
  
  return detect

# Get a color mask, in hsv color space
def get_color_mask(image,colorLower, colorUpper):
  
  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
  color_mask = cv2.inRange(hsv, colorLower, colorUpper)
  color_mask = cv2.dilate(color_mask, None, iterations=3)  
  return color_mask
  
# Just mix two mask  
def mix_mask(mask1,mask2):
  return cv2.bitwise_and(mask1, mask1, mask=mask2)
